<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/logo.png"/>

# Tutorial

This notebook gets you started with using
[Text-Fabric](https://annotation.github.io/text-fabric/) for coding in the Fusus al Hikam by Ibn Arabi, Lakhnawi edition.

Familiarity with the underlying
[data model](https://annotation.github.io/text-fabric/tf/about/datamodel.html)
is recommended.

## Installing Text-Fabric

See [here](https://annotation.github.io/text-fabric/tf/about/install.html)

## Tip
If you start computing with this tutorial, first copy its parent directory to somewhere else,
outside your repository.
If you pull changes from the repository later, your work will not be overwritten.
Where you put your tutorial directory is up to you.
It will work from any directory.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections

In [3]:
from tf.app import use

## Fusus data

Text-Fabric will fetch a standard set of features for you from the newest GitHub release binaries.

The data will be stored in the `text-fabric-data` in your home directory.

# Load Features
The data of the corpus is organized in features.
They are *columns* of data.
Think of the text as a gigantic spreadsheet, where row 1 corresponds to the
first word, row 2 to the second word, and so on, for all 50,000+ words.

The letters of each word is a column `form` in that spreadsheet.

The corpus contains ca. 35 columns, not only for the words, but also for
textual objects, such as *pieces*, *pages*, and *lines*.

Instead of putting that information in one big table, the data is organized in separate columns.
We call those columns **features**.

For the very last version, use `hot`.

For the latest release, use `latest`.

If you have cloned the repos (TF app and data), use `clone`.

If you do not want/need to upgrade, leave out the checkout specifiers.

In [4]:
A = use("among/fusus", hoist=globals())

This is Text-Fabric 9.2.2
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

37 features found and 0 ignored


## API

At this point it is helpful to throw a quick glance at the text-fabric API documentation
(see the links under **API Members** above).

The most essential thing for now is that we can use `F` to access the data in the features
we've loaded.
But there is more, such as `N`, which helps us to walk over the text, as we see in a minute.

# Counting

In order to get acquainted with the data, we start with the simple task of counting.

## Count all nodes
We use the
[`N.walk()` generator](https://annotation.github.io/text-fabric/tf/core/nodes.html#tf.core.nodes.Nodes.walk)
to walk through the nodes.

We compared corpus to a gigantic spreadsheet, where the rows correspond to the words.
In Text-Fabric, we call the rows `slots`, because they are the textual positions that can be filled with words.

We also mentioned that there are also more textual objects.
They are the verses, chapters and books.
They also correspond to rows in the big spreadsheet.

In Text-Fabric we call all these rows *nodes*, and the `N()` generator
carries us through those nodes in the textual order.

Just one extra thing: the `info` statements generate timed messages.
If you use them instead of `print` you'll get a sense of the amount of time that
the various processing steps typically need.

In [5]:
A.indent(reset=True)
A.info("Counting nodes ...")

i = 0
for n in N.walk():
    i += 1

A.info("{} nodes".format(i))

  0.00s Counting nodes ...
  0.01s 57143 nodes


## What are those nodes?
Every node has a type, like word, or phrase, sentence.
We know that we have approximately 100,000 words and a few other nodes.
But what exactly are they?

Text-Fabric has two special features, `otype` and `oslots`, that must occur in every Text-Fabric data set.
`otype` tells you for each node its type, and you can ask for the number of `slot`s in the text.

Here we go!

In [6]:
F.otype.slotType

'word'

In [7]:
F.otype.maxSlot

40983

In [8]:
F.otype.maxNode

57143

In [9]:
F.otype.all

('piece', 'page', 'sentence', 'line', 'column', 'span', 'word')

In [10]:
C.levels.data

(('piece', 1413.2068965517242, 50215, 50243),
 ('page', 101.69478908188586, 49812, 50214),
 ('sentence', 16.789430561245393, 50244, 52684),
 ('line', 9.380407415884642, 45443, 49811),
 ('column', 9.191074231890559, 40984, 45442),
 ('span', 9.191074231890559, 52685, 57143),
 ('word', 1, 1, 40983))

This is interesting: above you see all the textual objects, with the average size of their objects,
the node where they start, and the node where they end.

## Count individual object types
This is an intuitive way to count the number of nodes in each type.
Note in passing, how we use the `indent` in conjunction with `info` to produce neat timed
and indented progress messages.

In [11]:
A.indent(reset=True)
A.info("counting objects ...")

for otype in F.otype.all:
    i = 0
    A.indent(level=1, reset=True)

    for n in F.otype.s(otype):
        i += 1

    A.info("{:>7} {}s".format(i, otype))

A.indent(level=0)
A.info("Done")

  0.00s counting objects ...
   |     0.00s      29 pieces
   |     0.00s     403 pages
   |     0.00s    2441 sentences
   |     0.00s    4369 lines
   |     0.00s    4459 columns
   |     0.00s    4459 spans
   |     0.00s   40983 words
  0.01s Done


# Viewing textual objects

We use the A API (the extra power) to peek into the corpus.

Let's inspect some words.

In [12]:
wordShow = (1000, 10000, 50000)
for word in wordShow:
    A.pretty(word, withNodes=True)

# Feature statistics

`F`
gives access to all features.
Every feature has a method
`freqList()`
to generate a frequency list of its values, higher frequencies first.
Here are the words in beta code, a top 20:

In [13]:
F.lettersn.freqList()[0:20]

(('fy', 1341),
 ('mn', 1181),
 ('mā', 665),
 ('āllh', 619),
 ('', 604),
 ('lā', 486),
 ('ʿlá', 466),
 ('ān', 452),
 ('ālā', 420),
 ('ʿlyh', 322),
 ('hw', 311),
 ('lh', 294),
 ('ʿn', 290),
 ('ālḥḳ', 280),
 ('ḏlk', 279),
 ('hḏā', 273),
 ('bh', 257),
 ('kān', 249),
 ('whw', 244),
 ('wlā', 243))

## Word distribution

Let's do a bit more fancy word stuff.

### Hapaxes

A hapax can be found by inspecting words and see how many occurrences they have.
If that is number is one, we have a hapax.

We work with the ASCII transliteration.

We print 10 hapaxes with their gloss.

In [14]:
A.indent(reset=True)

hapax = []
lexIndex = collections.defaultdict(list)

for n in F.otype.s("word"):
    lexIndex[F.lettersp.v(n)].append(n)

hapax = dict((lex, occs) for (lex, occs) in lexIndex.items() if len(occs) == 1)

A.info("{} hapaxes found".format(len(hapax)))

for h in sorted(hapax)[0:10]:
    print(f"\t{h}")

  0.08s 5731 hapaxes found
	'_alh=t
	'_alhtkm
	'_amnw_a
	'_ant
	'_at/akm
	'_atyn_a
	'_ay=t
	'_aytn_a
	*d_a*hk=t
	*d_a*k


If we want more info on the hapaxes, we get that by means of its *node*.
The `lexIndex` dictionary stores the occurrences of a lexeme as a list of nodes.

Let's get the latin beta code and the arabic form of those 10 hapaxes.

In [15]:
for h in sorted(hapax)[0:10]:
    node = hapax[h][0]
    print(f"\t{F.lettersn.v(node):<12} {F.letters.v(node)}")

	ʾālhŧ        ءالهة
	ʾālhtkm      ءالهتكم
	ʾāmnwā       ءامنوا
	ʾānt         ءأنت
	ʾātákm       ءاتىكم
	ʾātynā       ءاتينا
	ʾāyŧ         ءاية
	ʾāytnā       ءايتنا
	ḍāḥkŧ        ضاحكة
	ḍāḳ          ضاق


### Small occurrence base

The occurrence base of a word are the pieces in which it occurs.
Let's look for words that occur in a single piece.

Oh yes, we have already found the hapaxes, we will skip them here.

In [16]:
A.indent(reset=True)
A.info("Finding single piece words")

lexPieceIndex = {}
allPiece = collections.defaultdict(set)
for p in F.otype.s("piece"):
    for w in L.d(p, otype="word"):
        allPiece[p].add(F.lettersp.v(w))

for (lex, occs) in lexIndex.items():
    lexPieceIndex[lex] = set(L.u(n, otype="piece")[0] for n in occs)

singlePieceLex = collections.defaultdict(set)
for (lex, pieces) in lexPieceIndex.items():
    if len(pieces) == 1:
        singlePieceLex[list(pieces)[0]].add(lex)

singlePiece = {piece: len(lexs) for (piece, lexs) in singlePieceLex.items()}

A.info("found {} single piece words".format(sum(singlePiece.values())))

  0.00s Finding single piece words
  0.19s found 6408 single piece words


In [17]:
print(
    "{:<20}{:>5}{:>5}{:>5}\n{}".format(
        "piece",
        "#all",
        "#own",
        "%own",
        "-" * 35,
    )
)
pieceList = []

for p in F.otype.s("piece"):
    piece = T.pieceName(p)
    a = len(allPiece[p])
    o = singlePiece.get(p, 0)
    pc = 100 * o / a
    pieceList.append((piece, a, o, pc))

for x in sorted(pieceList, key=lambda e: (-e[3], -e[1], e[0])):
    print("{:<20} {:>4} {:>4} {:>4.1f}%".format(*x))

piece                #all #own %own
-----------------------------------
3                     201   89 44.3%
28                   1627  672 41.3%
18                   1239  482 38.9%
30                   1242  465 37.4%
6                     837  311 37.2%
27                    681  243 35.7%
11                   1123  394 35.1%
13                    954  333 34.9%
14                    418  137 32.8%
19                   1010  329 32.6%
21                    432  139 32.2%
5                    1026  330 32.2%
25                    770  239 31.0%
16                    603  185 30.7%
23                    323   97 30.0%
8                     471  140 29.7%
9                     666  195 29.3%
22                    565  164 29.0%
20                    735  212 28.8%
10                    427  123 28.8%
15                    785  226 28.8%
4                     936  269 28.7%
17                    764  219 28.7%
7                     565  157 27.8%
26                    448  119 26.6%
24 

# Layer API
We travel upwards and downwards, forwards and backwards through the nodes.
The Layer-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow of precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible node types.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result are ordered according to the order of things in the text.

The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first word to the piece it contains.
Note the `[0]` at the end. You expect one piece, yet `L` returns a tuple.
To get the only element of that tuple, you need to do that `[0]`.

If you are like me, you keep forgetting it, and that will lead to weird error messages later on.

In [18]:
firstPiece = L.u(1, otype="piece")[0]
print(firstPiece)

50215


# Text API

So far, we have mainly seen nodes and their numbers, and the names of node types.
You would almost forget that we are dealing with text.
So let's try to see some text.

In the same way as `F` gives access to feature data,
`T` gives access to the text.
That is also feature data, but you can tell Text-Fabric which features are specifically
carrying the text, and in return Text-Fabric offers you
a Text API: `T`.

## Formats
Syriac text can be represented in a number of ways:

* in transliteration, or in Syriac characters,
* showing the actual text or only the lexemes,

If you wonder where the information about text formats is stored:
not in the program text-fabric, but in the data set.
It has a feature `otext`, which specifies the formats and which features
must be used to produce them. `otext` is the third special feature in a TF data set,
next to `otype` and `oslots`.
It is an optional feature.
If it is absent, there will be no `T` API.

Here is a list of all available formats in this data set.

In [19]:
sorted(T.formats)

['text-afifi-full',
 'text-afifi-nice',
 'text-afifi-plain',
 'text-afifi-trans',
 'text-orig-full',
 'text-orig-nice',
 'text-orig-plain',
 'text-orig-trans']

## Using the formats

We can pretty display in other formats:

In [20]:
for word in wordShow:
    A.pretty(word, fmt="text-orig-trans")

Now let's use those formats to print out the first verse of the corpus.

In [21]:
for fmt in sorted(T.formats):
    print("{}:\n\t{}".format(fmt, T.text(range(1, 12), fmt=fmt)))

text-afifi-full:
	بنزليلالع يلرعاالحمد له منزل الحك على قلوب الكلم بأحدية الطريق 
text-afifi-nice:
	bnzlylālʿ ylrʿāālḥmd lh mnzl ālḥk ʿlá ḳlwb ālklm bāḥdyŧ ālṭryḳ 
text-afifi-plain:
	bnzlyl_al` ylr`_a_al*hmd lh mnzl _al*hk `l/a *klwb _alklm b_a*hdy=t _al*try*k 
text-afifi-trans:
	bnzlylāl` ylr`āālḥmd lh mnzl ālḥk `lā qlwb ālklm bāḥdyt ālṭryq 
text-orig-full:
	  الحمد لله منزل الحكم على قلوب الكلم بأحدية الطريق 
text-orig-nice:
	  ālḥmd llh mnzl ālḥkm ʿlá ḳlwb ālklm bāḥdyŧ ālṭryḳ 
text-orig-plain:
	  _al*hmd llh mnzl _al*hkm `l/a *klwb _alklm b_a*hdy=t _al*try*k 
text-orig-trans:
	  ālḥmd llh mnzl ālḥkm `lā qlwb ālklm bāḥdyt ālṭryq 


If we do not specify a format, the **default** format is used (`text-orig-full`).

In [22]:
print(T.text(range(1, 12)))

  الحمد لله منزل الحكم على قلوب الكلم بأحدية الطريق 


## Whole text in all formats in less than a second
Part of the pleasure of working with computers is that they can crunch massive amounts of data.
The text of this corpus is a piece of cake.

It takes less than a second to have that cake and eat it.

In [23]:
A.indent(reset=True)
A.info("writing plain text of the Fusus in all formats")

text = collections.defaultdict(list)

for v in F.otype.s("page"):
    words = L.d(v, "word")
    for fmt in sorted(T.formats):
        text[fmt].append(T.text(words, fmt=fmt))

A.info("done {} formats".format(len(text)))

for fmt in sorted(text):
    print("{}\n{}\n".format(fmt, "\n".join(text[fmt][0:5])))

  0.00s writing plain text of the Fusus in all formats
  0.56s done 8 formats
text-afifi-full
بنزليلالع يلرعا
الحمد له منزل الحك على قلوب الكلم بأحدية الطريق الأمم من المقام االأقدم وإن اختلفت النحل والملل لاختلاف الأمم 
وصلى الله على مد الهمم من خزائن الجود والكرم بالقيل الأقوم محمد وعلى آله وسلم أما بعد فإني رأيت رسول الله صلى الله عليه وسلم في مبشرة أريتها فيالعشر الآخر من محرم سنة سبع وعشرون وستمائة بمحروسة دمشق وبيدهصلى الله عليه وسلم كتاب فقال لي هذا تاب فصوص الحكم» خذه واخرجبه إلى الناس ينتفعون به 
فقلت المع والطاعة له ولرسوله وأولي الأمر مناكما أمرنا فحققنت الأمنية وأخلصت النية وجردت القصد والممة إلىإبراز هذا الكتاب كما حده لي رسول الله صلى الله عليه وسلم من غير زيادةولا نقصان وسألت الله تعالى أن يجعلني فيه وفي جميع أحوالي منعباده الذين ليس للشيطان عليهم سطان وأن يخصني في جميع ما يرقمهبناني وينطق به لساني وينطوي عليه جناني بالالقاء السثبوحي والنفثالروحي في 
الروع النفسي بالتأييد الاعتصامي حتى أكون مترجما لامتحكما ليتحقق من يقف عليه من أهل الله أصحاب القلوبأنه من مقام التقديس ال

### The full plain text
We write a few formats to file, in your `Downloads` folder.

In [24]:
orig = "text-orig-full"
trans = "text-orig-trans"
for fmt in (orig, trans):
    with open(os.path.expanduser(f"~/Downloads/FususL-{fmt}.txt"), "w") as f:
        f.write("\n".join(text[fmt]))

In [25]:
!head -n 20 ~/Downloads/FususL-{orig}.txt

  
الحمد لله منزل الحكم على قلوب الكلم بأحدية الطريق الأمم من المقام الأقدم  وإن اختلفت النحل والملل لاختلاف الأمم. 
وصلى الله على ممد الهمم، من خزائن الجود والكرم، بالقيل الأقوم، محمد وعلى آله وسلم. أما بعد: فإني رأيت رسول الله صلى الله عليه وسلم في مبشرة أريتها في العشر الأخر من المحرم سنة سبع وعشرين وستمائة بمحروسة دمشق، وبيده صلى الله عليه وسلم كتاب. فقال لي: هذا كتاب فصوص الحكم. خذه واخرج به إلى الناس ينتفعون به». 
فقلت: السمع والطاعة لله ولرسوله وأولي الأمر منا، كما أمرنا». فحققت الأمنية، وأخلصت النية، وجردت القصد والهمة إلى إبراز هذا الكتاب كما حده لي رسول الله صلى الله عليه وسلم من غير زيادة ولا نقصان. وسألت الله  أن يجعلني فيه وفي جميع أحوالي من عباده الذين ليس للشيطان عليهم سلطان، وأن يخصني في جميع ما يرقمه بناني، وينطق به لساني، وينطوي عليه جناني، بالإلقاء السبوحي والنفث الروحي في 
الروع النفسي بالتأييد الاعتصامي، حتى أكون مترجما لا متحكما، ليتحقق من يقف عليه من أهل الله، أصحاب القلوب، أنه من مقام التقديس المنزه عن الأغراض النفسية التي يدخلها التلبيس. وأرجو أن يكون الحق تعال

## Sections

A section is a piece, a page or a line.
Knowledge of sections is not baked into Text-Fabric.
The config feature `otext.tf` may specify three section levels, and tell
what the corresponding node types and features are.

From that knowledge it can construct mappings from nodes to sections, e.g. from verse
nodes to tuples of the form:

    (piece number, page number, line number)

Here are examples of getting the section that corresponds to a node and vice versa.

**NB:** `sectionFromNode` always delivers a verse specification, either from the
first slot belonging to that node, or, if `lastSlot`, from the last slot
belonging to that node.

In [26]:
for x in (
    ("section of first word", T.sectionFromNode(1)),
    ("node of piece 8, page 88, line 3", T.nodeFromSection((8, 88, 3))),
):
    print("{:<30} {}".format(*x))

section of first word          (1, 0, 0)
node of piece 8, page 88, line 3 46293


# Next steps

By now you have an impression how to compute around in the text.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

## Search
Text-Fabric contains a flexible search engine, that does not only work for this data,
but also for data that you add to it.
There are tutorials dedicated to search, but not in this corpus.
See e.g. [quran search](https://nbviewer.org/github/q-ran/quran/blob/master/tutorial/search.ipynb).

## Add your own data
If you study the additional data, you can observe how that data is created and also
how it is turned into a text-fabric data module.
The last step is incredibly easy. You can write out every Python dictionary where the keys are numbers
and the values string or numbers as a Text-Fabric feature.
When you are creating data, you have already constructed those dictionaries, so writing
them out is just one method call.

You can then easily share your new features on GitHub, so that your colleagues everywhere
can try it out for themselves.

---

CC-BY Dirk Roorda